# Évaluation qualitative
> Comparons les différents algorithmes en terme de qualité de prédiction et de temps de calcul

# Comparaison entre algorithmes

Charger les bibliothèques et les données

In [1]:
%autosave 0
%reload_ext autoreload
%autoreload 2

from data import *
import popularity
import knn
import knn_item_based
import svd
import als
from svd import *
from eval import *

Autosave disabled


In [2]:
import sys
sys.path
import data

In [3]:
M = load_data(tiny=True)
M.shape, np.sum(~np.isnan(M))

((50, 40), 389)

Liste d'algorithmes à comparer

In [4]:
# recommenders = [
#     {"fn": popularity.complete, "label": "popularity"},
#     {"fn": lambda M_train: knn.complete(M_train, k=10), "label": "knn10"},
#     {"fn": lambda M_train: svd.complete(M_train, k=10), "label": "svd0_10"},
#     {"fn": lambda M_train: svd.complete(M_train, k=20), "label": "svd0_20"},
#     {"fn": lambda M_train: svd.complete(M_train, k=10, replaceNA_fn=replaceNA_with_mean), "label": "svdMean_10"},
#     {"fn": lambda M_train: svd.complete(M_train, k=20, replaceNA_fn=replaceNA_with_mean), "label": "svdMean_20"},
#     {"fn": lambda M_train: als.complete(M_train, k=10, lambd=10**-1, n_iter=5), "label": "als10"},
#     {"fn": lambda M_train: als.complete(M_train, k=40, lambd=10**-1, n_iter=5), "label": "als40"},
#     {"fn": lambda M_train: complete_softImpute(M_train, lambd=10**+1, n_iter=20), "label": "softImpute10"},
#     {"fn": lambda M_train: complete_softImpute(M_train, lambd=10**0, n_iter=20), "label": "softImpute1"},
#     {"fn": lambda M_train: complete_softImpute(M_train, lambd=10**-1, n_iter=20), "label": "softImpute0_1"},
#     ]

Comparaison

# Recherche de meilleiur valeur de K du KNN

In [5]:
# Tester plusieurs valeurs de k pour KNN
k_values = list(range(2, 30))
recommenders = [
    {"fn": lambda M_train, k=k: knn.complete(M_train, k=k), "label": f"knn{k}"} 
    for k in k_values
]

# Comparaison des performances
res_knn_k = quantitative_comparison(RMSE, M, recommenders, prop=0.8, nrep=4)

# Trouver les meilleures valeur de k
best_k = res_knn_k.nsmallest(5, 'validation score')


# Sauvegarde des résultats
best_k.to_csv("best_knn_result.csv", index=False)



In [6]:
# Affichage des résultats
best_k.head()

,recommender,validation score,training score,computation time
14,knn16,1.075089,0.0,0.640961
11,knn13,1.075189,0.0,0.654125
15,knn17,1.075297,0.0,0.824311
13,knn15,1.075620,0.0,0.786097
10,knn12,1.076069,0.0,0.855000


# Recherche de meilleiur valeur de K du KNN Item Based

In [7]:

recommenders = [
    {"fn": lambda M_train, k=k: knn_item_based.complete_item_based(M_train, k=k), "label": f"knn_item{k}"} 
    for k in k_values
]

# Comparaison des performances
res_knn_item_k = quantitative_comparison(RMSE, M, recommenders, prop=0.8, nrep=4)

# Trouver les meilleures valeur de k
best_k_knn_item = res_knn_item_k.nsmallest(5, 'validation score')


# Sauvegarde des résultats
best_k_knn_item.to_csv("best_knn_item_result.csv", index=False)

In [8]:
best_k_knn_item.head()

,recommender,validation score,training score,computation time
27,knn_item29,1.044027,0.0,0.820268
26,knn_item28,1.044084,0.0,0.721500
25,knn_item27,1.044196,0.0,0.447948
23,knn_item25,1.044271,0.0,0.450590
24,knn_item26,1.044482,0.0,0.622691


# Recherche de meilleiur valeur de K du SVD

In [9]:
# Tester plusieurs valeurs de k pour KNN
recommenders = [
    {"fn": lambda M_train, k=k: svd.complete(M_train, k=k), "label": f"svd{k}"} 
    for k in k_values
]

# Comparaison des performances
res_svd_k = quantitative_comparison(RMSE, M, recommenders, prop=0.8, nrep=4)

# Trouver les meilleures valeur de k
best_k_svd = res_svd_k.nsmallest(5, 'validation score')


# Sauvegarde des résultats
best_k_svd.to_csv("best_svd_result.csv", index=False)

In [10]:
best_k_svd.head(50)

,recommender,validation score,training score,computation time
0,svd2,3.073598,2.408178,0.005718
1,svd3,3.264302,2.224919,0.000000
2,svd4,3.353896,2.039790,0.000000
3,svd5,3.420154,1.862982,0.000000
4,svd6,3.489354,1.702019,0.000000


# Imputation par la moyenne

In [11]:
# Tester plusieurs valeurs de k pour KNN
recommenders = [
    {"fn": lambda M_train, k=k: svd.complete(M_train, k=k,replaceNA_fn=replaceNA_with_mean), "label": f"svd{k}"} 
    for k in k_values
]

# Comparaison des performances
res_svd_k = quantitative_comparison(RMSE, M, recommenders, prop=0.8, nrep=4)

# Trouver les meilleures valeur de k
best_k_svd = res_svd_k.nsmallest(5, 'validation score')


# Sauvegarde des résultats
best_k_svd.to_csv("best_svd_Imput_moy.csv", index=False)

c:\Users\SobjioLagnol(Externa\OneDrive - Majorel\Bureau\SystRecomandation\svd.py:21: RuntimeWarning: Mean of empty slice
  col_mean = np.nanmean(M_train, axis=0)


In [12]:
best_k_svd.head(50)

,recommender,validation score,training score,computation time
0,svd2,1.181793,0.309941,0.003669
22,svd24,1.182905,0.032138,0.000000
11,svd13,1.182997,0.115966,0.000000
24,svd26,1.183266,0.022661,0.000000
12,svd14,1.183382,0.105476,0.000000


l'imputation pas la moyenne est meilleur 

# Recherche de meilleiur valeur de K pour ALS

In [13]:
# Tester plusieurs valeurs de k pour KNN
recommenders = [
    {"fn": lambda M_train, k=k: als.complete(M_train, k=k), "label": f"svd{k}"} 
    for k in k_values
]

# Comparaison des performances
res_als_k = quantitative_comparison(RMSE, M, recommenders, prop=0.8, nrep=4)

# Trouver les meilleures valeur de k
best_k_als = res_als_k.nsmallest(5, 'validation score')


# Sauvegarde des résultats
best_k_als.to_csv("best_als_result.csv", index=False)

In [14]:
best_k_als.head(50)

,recommender,validation score,training score,computation time
19,svd21,1.289165,0.028175,0.170036
27,svd29,1.322864,0.035901,0.260192
15,svd17,1.338270,0.026912,0.021820
13,svd15,1.341688,0.024003,0.031311
21,svd23,1.344799,0.031009,0.136568


# 4  Autres Metriques d'evaluation 

#### **1. Description des métriques et protocole d'évaluation**  

- **MAE (Mean Absolute Error)** :  
  - Objectif : Mesure l'erreur absolue moyenne entre les prédictions et les valeurs réelles.  
  - Protocole d'évaluation :  
    1. Séparer les données en ensemble d'entraînement et de validation.  
    2. Appliquer l’algorithme de recommandation pour compléter la matrice d’évaluation.  
    3. Comparer les prédictions aux valeurs réelles en calculant la moyenne des erreurs absolues.  
    4. Une faible valeur de MAE indique de meilleures performances.  

- **Precision k** :  
  - Objectif : Mesure la proportion d'items recommandés dans le top-k qui sont effectivement pertinents.  
  - Protocole d’évaluation :  
    1. Définir un ensemble de validation avec des interactions utilisateur-item connues.  
    2. Générer une liste de recommandations pour chaque utilisateur.  
    3. Comparer les k premiers items recommandés avec la liste des items pertinents.  
    4. Calculer la proportion d’items corrects dans le top-k recommandé.  
    5. La précision est élevée si la majorité des recommandations sont pertinentes.  

- **Recall k** :  
  - Objectif : Évalue la capacité du système à recommander tous les items pertinents.  
  - Protocole d’évaluation :  
    1. Identifier la liste des items pertinents pour un utilisateur.  
    2. Vérifier combien d’items pertinents apparaissent dans le top-k recommandé.  
    3. Calculer la proportion d’items pertinents retrouvés sur l’ensemble des items pertinents existants.  
    4. Un rappel élevé indique que le modèle capte bien les préférences des utilisateurs.  

---

#### **2. Dépendance des valeurs de précision et de rappel au nombre d'items recommandés (k)**  
- Le choix de **k** influence directement ces métriques :  
  - Si **k est trop petit**, la précision peut être élevée, mais le rappel sera faible.  
  - Si **k est trop grand**, le rappel peut être élevé, mais la précision risque de baisser.  
- L’évaluation doit être faite avec différents k pour comparer l'impact sur la performance des modèles.  

---

#### **3. Influence du choix du "candidate set"**  
- Le **candidate set** représente l’ensemble des items parmi lesquels les recommandations sont sélectionnées.  
- Différentes stratégies peuvent être utilisées :  
  - **Tous les items disponibles** : Peut biaiser l'évaluation si le dataset est très grand.  
  - **Items déjà vus par l’utilisateur** : Peut donner un avantage aux modèles basés sur l’historique.  
  - **Items populaires ou filtrés** : Peut favoriser les recommandations triviales.  
- Comparer les résultats selon différentes méthodes de sélection permet de mieux comprendre les forces et faiblesses des algorithmes.  



In [15]:
def best_recommender(df):
    metrics = df.columns[1:]  # Exclure la colonne "recommender"
    best_models = []

    for metric in metrics:
        if metric in ["RMSE (validation)", "MAE (validation)", "computation time"]:  
            # Pour ces métriques, on cherche la valeur minimale
            best_row = df.loc[df[metric].idxmin()]
        else:  
            # Pour les autres métriques, on cherche la valeur maximale
            best_row = df.loc[df[metric].idxmax()]
        
        best_models.append({"Metric": metric, "Best Recommender": best_row["recommender"], "Best Value": best_row[metric]})

    return pd.DataFrame(best_models)


In [16]:
# Tester plusieurs valeurs de k pour KNN
k_values = list(range(2, 50))

scoring_fns = [RMSE, MAE, precision_at_k, recall_at_k, user_space_coverage, item_space_coverage, ranking_based_on_ratings]



In [17]:
recommenders = [
    {"fn": lambda M_train, k=k: knn.complete(M_train, k=k), "label": f"knn{k}"} 
    for k in k_values
]

df_final = None 
for scoring_fn in scoring_fns:
    df_temp = quantitative_comparison(scoring_fn, M, recommenders, prop=0.8, nrep=3)
    df_temp = df_temp.rename(columns={'validation score': f'{scoring_fn.__name__} (validation)'})
    
    if df_final is None:
        df_final = df_temp
    else:
        df_final = df_final.merge(df_temp[['recommender', f'{scoring_fn.__name__} (validation)']], on='recommender')


best_models = best_recommender(df_final)
best_models

,Metric,Best Recommender,Best Value
0,RMSE (validation),knn16,1.133547
1,training score,knn2,0.000000
2,computation time,knn6,0.557267
3,MAE (validation),knn16,0.815690
4,precision_at_k (validation),knn11,0.065333
5,recall_at_k (validation),knn6,0.067333
6,user_space_coverage (validation),knn2,1.000000
7,item_space_coverage (validation),knn2,1.000000
8,ranking_based_on_ratings (validation),knn7,0.017346


In [18]:


# Sauvegarde des résultats
best_models.to_csv("Metrique_KNN_K.csv", index=False)

In [19]:
# Tester plusieurs valeurs de k pour KNN
recommenders = [
    {"fn": lambda M_train, k=k: knn_item_based.complete_item_based(M_train, k=k), "label": f"knn_item{k}"} 
    for k in k_values
]
df_final = None 
for scoring_fn in scoring_fns:
    df_temp = quantitative_comparison(scoring_fn, M, recommenders, prop=0.8, nrep=3)
    df_temp = df_temp.rename(columns={'validation score': f'{scoring_fn.__name__} (validation)'})
    
    if df_final is None:
        df_final = df_temp
    else:
        df_final = df_final.merge(df_temp[['recommender', f'{scoring_fn.__name__} (validation)']], on='recommender')




best_models = best_recommender(df_final)
best_models

,Metric,Best Recommender,Best Value
0,RMSE (validation),knn_item27,1.071630
1,training score,knn_item2,0.000000
2,computation time,knn_item41,0.358192
3,MAE (validation),knn_item17,0.851445
4,precision_at_k (validation),knn_item2,0.042667
5,recall_at_k (validation),knn_item2,0.046667
6,user_space_coverage (validation),knn_item2,1.000000
7,item_space_coverage (validation),knn_item2,1.000000
8,ranking_based_on_ratings (validation),knn_item2,0.253023


In [20]:
# Sauvegarde des résultats
best_models.to_csv("Metrique_KNN_K_Item.csv", index=False)


In [21]:
# Tester plusieurs valeurs de k pour KNN
recommenders = [
    {"fn": lambda M_train, k=k: svd.complete(M_train, k=k), "label": f"svd{k}"} 
    for k in k_values
]
df_final = None 
for scoring_fn in scoring_fns:
    df_temp = quantitative_comparison(scoring_fn, M, recommenders, prop=0.8, nrep=3)
    df_temp = df_temp.rename(columns={'validation score': f'{scoring_fn.__name__} (validation)'})
    
    if df_final is None:
        df_final = df_temp
    else:
        df_final = df_final.merge(df_temp[['recommender', f'{scoring_fn.__name__} (validation)']], on='recommender')




best_models = best_recommender(df_final)
best_models

,Metric,Best Recommender,Best Value
0,RMSE (validation),svd2,3.095579
1,training score,svd2,2.415383
2,computation time,svd3,0.000000
3,MAE (validation),svd2,2.849935
4,precision_at_k (validation),svd2,0.082667
5,recall_at_k (validation),svd2,0.077333
6,user_space_coverage (validation),svd2,1.000000
7,item_space_coverage (validation),svd2,1.000000
8,ranking_based_on_ratings (validation),svd5,0.085546


In [22]:
# Sauvegarde des résultats
best_models.to_csv("Metrique_SVD_K.csv", index=False)


In [23]:
recommenders = [
    {"fn": lambda M_train, k=k: als.complete(M_train, k=k), "label": f"als_{k}"} 
    for k in k_values
]
df_final = None 
for scoring_fn in scoring_fns:
    df_temp = quantitative_comparison(scoring_fn, M, recommenders, prop=0.8, nrep=3)
    df_temp = df_temp.rename(columns={'validation score': f'{scoring_fn.__name__} (validation)'})
    
    if df_final is None:
        df_final = df_temp
    else:
        df_final = df_final.merge(df_temp[['recommender', f'{scoring_fn.__name__} (validation)']], on='recommender')




best_models = best_recommender(df_final)
best_models


,Metric,Best Recommender,Best Value
0,RMSE (validation),als_2,1.361370
1,training score,als_2,0.578261
2,computation time,als_2,0.019460
3,MAE (validation),als_2,1.033093
4,precision_at_k (validation),als_48,0.086667
5,recall_at_k (validation),als_36,0.088667
6,user_space_coverage (validation),als_2,1.000000
7,item_space_coverage (validation),als_2,1.000000
8,ranking_based_on_ratings (validation),als_40,0.074912


In [24]:
# Sauvegarde des résultats
best_models.to_csv("Metrique_ALS_K.csv", index=False)


# Paramétrage de méthode basée sur SVD

Charger les bibliothèques et les données

In [25]:
M = load_data(tiny=False)
M.shape, np.sum(~np.isnan(M))

((500, 400), 31553)

On fait varier $k$. Observe-t-on un phénomène de sur-apprentissage ?

In [ ]:
recommenders = [
   {"fn": lambda M_train: knn.complete(M_train, k=10), "label": "knn10"},
    {"fn": lambda M_train: svd.complete(M_train, k=10), "label": "svd10"},
    {"fn": lambda M_train: svd.complete(M_train, k=20), "label": "svd20"},
    {"fn": lambda M_train: svd.complete(M_train, k=50), "label": "svd50"},
    {"fn": lambda M_train: svd.complete(M_train, k=100), "label": "svd100"},
    {"fn": lambda M_train: svd.complete(M_train, k=200), "label": "svd200"},
    {"fn": lambda M_train: svd.complete(M_train, k=400), "label": "svd400"},
    ]
res_fill_with_zeros = quantitative_comparison(RMSE, M, recommenders, prop=0.8, nrep=4)
res_fill_with_zeros

Oui, on observe un phénomène de sur-apprentissage (overfitting), en particulier sur les modèles SVD avec un nombre élevé de facteurs latents.